In [ ]:
#default_exp data.source

In [ ]:
from fastai_local.imports import *
from fastai_local.test import *
from fastai_local.core import *
from fastai_local.data.pipeline import *

## DataSource -

In [ ]:
# export
class DataSource():
    "Applies a `Pipeline` of `tfms` to filtered subsets of `items`"
    def __init__(self, items, tfms=noop, filts=None):
        if filts is None: filts = [range_of(items)]
        ft = mask2idxs if isinstance(filts[0][0], bool) else listify
        self.filts = listify(ft(filt) for filt in filts)
        self.items,self.tfm = listify(items),Pipeline(tfms)
        self.tfm.setup(self)

    def __len__(self): return len(self.filts)
    def len(self, filt=0): return len(self.filts[filt])
    def __getitem__(self, i): return FilteredList(self, i)
    def decode(self, o, filt=0, **kwargs): return self.tfm.decode(o, filt=filt, **kwargs)
    def decoded(self, idx, filt=0): return self.decode(self.get(idx,filt), filt)
    def __iter__(self): return (self[i] for i in range_of(self))
    
    def get(self, idx, filt=0):
        if hasattr(idx,'__len__') and getattr(idx,'ndim',1):
            # rank>0 collection
            if isinstance(idx[0],bool): idx = mask2idxs(idx)
            return [self.get(i,filt) for i in idx]  # index list
        it = self.items[self.filts[filt][idx]]
        return self.tfm(it, filt=filt)

    def __eq__(self,b):
        if not isinstance(b,DataSource): b = DataSource(b)
        return len(b) == len(self) and all(o==p for o,p in zip(self,b))

    def __repr__(self):
        res = f'{self.__class__.__name__}\n'
        for i,o in enumerate(self): res += f'{i}: {coll_repr(o)}'
        return res
    
    @property
    def train(self): return self[0]
    @property
    def valid(self): return self[1]

In [ ]:
show_doc(DataSource.__len__, a)

## TfmList -

In [ ]:
#export
class TfmList():
    def __init__(self, ttfms, tfm=noop):
        self.activ,self.ttfms = False,[Transforms(tfm) for tfm in listify(ttfms)]

    def __call__(self, o, **kwargs):
        if self.activ: return self.activ(o, **kwargs)
        return [t(o, **kwargs) for t in self.ttfms]
    
    def decode(self, o, **kwargs): return [t.decode(p, **kwargs) for p,t in zip(o,self.ttfms)]

    def setup(self, o):
        for tfm in self.ttfms:
            self.activ = tfm
            tfm.setup(o)
        self.activ=None
        
    def show(self, o, **kwargs): return show_xs(o, self.ttfms, **kwargs)
    def __repr__(self): return f'TfmList({self.ttfms})'
    
    @property
    def xt(self): return self.ttfms[0]
    @property
    def yt(self): return self.ttfms[1]